In [3]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import cv2
import face_recognition
import numpy as np
import requests
import tempfile
import os

def register_face_to_server(server_url, user_id, person_name, image_path):
    """
    Envia a imagem, user_id e o nome para o servidor usando multipart/form-data.
    """
    data = {'user_id': user_id, 'name': person_name}

    try:
        with open(image_path, 'rb') as image_file:
            files = {'image': image_file}
            response = requests.post(server_url, data=data, files=files)

        if response.status_code in [200, 201]:
            response_json = response.json()

            if 'message' in response_json:
                messagebox.showinfo("Cadastro", response_json['message'])
            
        else:
            messagebox.showerror("Erro", f"Falha ao cadastrar rosto: {response.text}")

    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao cadastrar o rosto: {str(e)}")

def detect_faces_from_webcam_and_send_to_server(server_register_url, server_recognize_url):
    net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        print("Não foi possível acessar a webcam.")
        return

    stop = False
    face_detected = False
    temp_image_path = None  # Para armazenar a imagem temporária capturada

    def on_close():
        nonlocal stop, temp_image_path
        stop = True
        print("Fechando a janela e liberando a câmera...")
        if video_capture.isOpened():
            video_capture.release()
        if temp_image_path and os.path.exists(temp_image_path):
            try:
                os.remove(temp_image_path)  # Remover o arquivo temporário
            except PermissionError:
                print(f"Erro: O arquivo '{temp_image_path}' ainda está em uso.")
        root.destroy()

    root = tk.Tk()
    root.title("Webcam ao Vivo")

    label = tk.Label(root)
    label.pack()

    frame_controls = tk.Frame(root)
    frame_controls.pack()

    btn_register = tk.Button(frame_controls, text="Cadastrar Rosto", command=lambda: open_registration_form(server_register_url))
    btn_register.pack(side=tk.LEFT, padx=5)
    btn_register["state"] = tk.DISABLED  # Desabilitar até que um rosto seja detectado

    btn_recognize = tk.Button(frame_controls, text="Reconhecer Rosto", command=lambda: recognize_face(server_recognize_url))
    btn_recognize.pack(side=tk.LEFT, padx=5)

    btn_resume = tk.Button(frame_controls, text="Retomar Webcam", command=lambda: resume_webcam())
    btn_resume.pack(side=tk.LEFT, padx=5)
    btn_resume["state"] = tk.DISABLED  # Desabilitar até que o feed seja pausado

    root.bind('<Control-q>', lambda event: on_close())
    root.protocol("WM_DELETE_WINDOW", on_close)

    def resume_webcam():
        nonlocal face_detected
        face_detected = False
        btn_register["state"] = tk.DISABLED
        btn_resume["state"] = tk.DISABLED
        update_frame()

    def update_frame():
        nonlocal face_detected, temp_image_path
        if stop or face_detected:
            return

        ret, frame = video_capture.read()
        if not ret:
            print("Falha ao capturar imagem da webcam")
            return

        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        face_locations = []
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                face_locations.append((startY, endX, endY, startX))
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_image = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=frame_image)

        label.imgtk = imgtk
        label.configure(image=imgtk)

        if len(face_locations) > 0:
            print("Rosto detectado!")
            face_detected = True

            _, temp_image_path = tempfile.mkstemp(suffix='.png')
            cv2.imwrite(temp_image_path, frame)
            os.close(_)  # Fechar o arquivo corretamente

            btn_register["state"] = tk.NORMAL
            btn_resume["state"] = tk.NORMAL
        else:
            label.after(10, update_frame)

    def open_registration_form(server_register_url):
        nonlocal temp_image_path

        def on_submit():
            user_id = entry_user_id.get()
            person_name = entry_name.get()
            if user_id and person_name:
                if temp_image_path:
                    register_face_to_server(server_register_url, user_id, person_name, temp_image_path)
                else:
                    messagebox.showerror("Erro", "Nenhuma imagem de rosto capturada para cadastrar.")
                top.destroy()
            else:
                messagebox.showerror("Erro", "Por favor, preencha todos os campos.")

        top = tk.Toplevel(root)
        top.title("Cadastro de Rosto")

        tk.Label(top, text="User ID:").pack()
        entry_user_id = tk.Entry(top)
        entry_user_id.pack()

        tk.Label(top, text="Nome:").pack()
        entry_name = tk.Entry(top)
        entry_name.pack()

        tk.Button(top, text="Cadastrar", command=on_submit).pack()

    def recognize_face(server_recognize_url):
        ret, frame = video_capture.read()
        if not ret:
            messagebox.showerror("Erro", "Falha ao capturar imagem.")
            return

        face_encodings = face_recognition.face_encodings(frame)
        if len(face_encodings) > 0:
            data_to_send = {'encodings': [encoding.tolist() for encoding in face_encodings]}
            try:
                response = requests.post(server_recognize_url, json=data_to_send)
                response_data = response.json()
                messagebox.showinfo("Reconhecimento", response_data.get("message", "Nenhum dado retornado."))
            except Exception as e:
                messagebox.showerror("Erro", f"Erro ao comunicar com o servidor: {e}")
        else:
            messagebox.showinfo("Reconhecimento", "Nenhum rosto detectado.")

    update_frame()
    root.mainloop()

# Testar a função com as URLs de cadastro e reconhecimento
server_register_url = 'http://127.0.0.1:5000/register'
server_recognize_url = 'http://127.0.0.1:5000/recognize'
detect_faces_from_webcam_and_send_to_server(server_register_url, server_recognize_url)


Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Fechando a janela e liberando a câmera...


In [1]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import cv2
import face_recognition
import numpy as np
import requests
import tempfile
import os

def register_face_to_server(server_url, user_id, person_name, image_path):
    """
    Envia a imagem, user_id e o nome para o servidor usando multipart/form-data.
    """
    data = {'user_id': user_id, 'name': person_name}

    try:
        with open(image_path, 'rb') as image_file:
            files = {'image': image_file}
            response = requests.post(server_url, data=data, files=files)

        if response.status_code in [200, 201]:
            response_json = response.json()

            if 'message' in response_json:
                messagebox.showinfo("Cadastro", response_json['message'])
            
        else:
            messagebox.showerror("Erro", f"Falha ao cadastrar rosto: {response.text}")

    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao cadastrar o rosto: {str(e)}")

def detect_faces_from_webcam_and_send_to_server(server_url):
    net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        print("Não foi possível acessar a webcam.")
        return

    stop = False
    face_detected = False
    temp_image_path = None  # Para armazenar a imagem temporária capturada

    def on_close():
        nonlocal stop, temp_image_path
        stop = True
        print("Fechando a janela e liberando a câmera...")
        if video_capture.isOpened():
            video_capture.release()
        if temp_image_path and os.path.exists(temp_image_path):
            try:
                os.remove(temp_image_path)  # Remover o arquivo temporário
            except PermissionError:
                print(f"Erro: O arquivo '{temp_image_path}' ainda está em uso.")
        root.destroy()

    root = tk.Tk()
    root.title("Webcam ao Vivo")

    label = tk.Label(root)
    label.pack()

    frame_controls = tk.Frame(root)
    frame_controls.pack()

    btn_register = tk.Button(frame_controls, text="Cadastrar Rosto", command=lambda: open_registration_form(server_url))
    btn_register.pack(side=tk.LEFT, padx=5)
    btn_register["state"] = tk.DISABLED  # Desabilitar até que um rosto seja detectado

    btn_recognize = tk.Button(frame_controls, text="Reconhecer Rosto", command=lambda: recognize_face(server_url))
    btn_recognize.pack(side=tk.LEFT, padx=5)

    btn_resume = tk.Button(frame_controls, text="Retomar Webcam", command=lambda: resume_webcam())
    btn_resume.pack(side=tk.LEFT, padx=5)
    btn_resume["state"] = tk.DISABLED  # Desabilitar até que o feed seja pausado

    root.bind('<Control-q>', lambda event: on_close())
    root.protocol("WM_DELETE_WINDOW", on_close)

    def resume_webcam():
        nonlocal face_detected
        face_detected = False
        btn_register["state"] = tk.DISABLED
        btn_resume["state"] = tk.DISABLED
        update_frame()

    def update_frame():
        nonlocal face_detected, temp_image_path
        if stop or face_detected:
            return

        ret, frame = video_capture.read()
        if not ret:
            print("Falha ao capturar imagem da webcam")
            return

        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        face_locations = []
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                face_locations.append((startY, endX, endY, startX))
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_image = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=frame_image)

        label.imgtk = imgtk
        label.configure(image=imgtk)

        if len(face_locations) > 0:
            print("Rosto detectado!")
            face_detected = True

            _, temp_image_path = tempfile.mkstemp(suffix='.png')
            cv2.imwrite(temp_image_path, frame)
            os.close(_)  # Fechar o arquivo corretamente

            btn_register["state"] = tk.NORMAL
            btn_resume["state"] = tk.NORMAL
        else:
            label.after(10, update_frame)

    def open_registration_form(server_url):
        nonlocal temp_image_path

        def on_submit():
            user_id = entry_user_id.get()
            person_name = entry_name.get()
            if user_id and person_name:
                if temp_image_path:
                    register_face_to_server(server_url, user_id, person_name, temp_image_path)
                else:
                    messagebox.showerror("Erro", "Nenhuma imagem de rosto capturada para cadastrar.")
                top.destroy()
            else:
                messagebox.showerror("Erro", "Por favor, preencha todos os campos.")

        top = tk.Toplevel(root)
        top.title("Cadastro de Rosto")

        tk.Label(top, text="User ID:").pack()
        entry_user_id = tk.Entry(top)
        entry_user_id.pack()

        tk.Label(top, text="Nome:").pack()
        entry_name = tk.Entry(top)
        entry_name.pack()

        tk.Button(top, text="Cadastrar", command=on_submit).pack()

    def recognize_face(server_url):
        ret, frame = video_capture.read()
        if not ret:
            messagebox.showerror("Erro", "Falha ao capturar imagem.")
            return

        face_encodings = face_recognition.face_encodings(frame)
        if len(face_encodings) > 0:
            data_to_send = {'encodings': [encoding.tolist() for encoding in face_encodings]}
            try:
                response = requests.post(server_url, json=data_to_send)
                response_data = response.json()
                messagebox.showinfo("Reconhecimento", response_data.get("message", "Nenhum dado retornado."))
            except Exception as e:
                messagebox.showerror("Erro", f"Erro ao comunicar com o servidor: {e}")
        else:
            messagebox.showinfo("Reconhecimento", "Nenhum rosto detectado.")

    update_frame()
    root.mainloop()

# Testar a função com as URLs de cadastro e reconhecimento
server_register_url = 'http://127.0.0.1:5000/register'
detect_faces_from_webcam_and_send_to_server(server_register_url)


Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Fechando a janela e liberando a câmera...
